In [1]:
import numpy as np
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
import folium

In [2]:
df = pd.read_json('./jobscraper/jobscraper/it_jobs.json')
df

,title,city,url,details
0,Викладач програмування (нічна зміна),Дистанційно,https://www.work.ua/jobs/5708935/,"{'salary': ['9000', '19500 '], 'company': 'Jus..."
1,Фахівець текстової підтримки,"Київ,",https://www.work.ua/jobs/5668550/,"{'salary': ['13000', '18000 '], 'company': 'Me..."
2,Менеджер з продажу електроніки (7 км) в інтерн...,"Одеса,",https://www.work.ua/jobs/5043547/,"{'salary': ['25000', '28000 '], 'company': 'Ав..."
3,Microsoft CRM Developer,Київ,https://www.work.ua/jobs/5613679/,"{'salary': None, 'company': 'Агропросперіс', '..."
4,Продавець-консультант техніки Apple,"Київ,",https://www.work.ua/jobs/4850818/,"{'salary': ['23000', '46000 '], 'company': 'IP..."
...,...,...,...,...
5898,Головний спеціаліст з інформаційних технологій,"Львів,",https://www.work.ua/jobs/5638578/,"{'salary': ['9880 '], 'company': 'Галицький ра..."
5899,SMM-менеджер,"Київ,",https://www.work.ua/jobs/5318458/,"{'salary': ['25000 '], 'company': 'Бережанськи..."
5900,Адміністратор соц. мереж,Київ,https://www.work.ua/jobs/5624384/,"{'salary': ['23000', '25000 '], 'company': 'De..."
5901,Game Designer,Дистанційно,https://www.work.ua/jobs/3360940/,"{'salary': None, 'company': 'Alta Games', 'com..."


In [3]:
df1 = df[['title', 'city', 'url']]
df1

,title,city,url
0,Викладач програмування (нічна зміна),Дистанційно,https://www.work.ua/jobs/5708935/
1,Фахівець текстової підтримки,"Київ,",https://www.work.ua/jobs/5668550/
2,Менеджер з продажу електроніки (7 км) в інтерн...,"Одеса,",https://www.work.ua/jobs/5043547/
3,Microsoft CRM Developer,Київ,https://www.work.ua/jobs/5613679/
4,Продавець-консультант техніки Apple,"Київ,",https://www.work.ua/jobs/4850818/
...,...,...,...
5898,Головний спеціаліст з інформаційних технологій,"Львів,",https://www.work.ua/jobs/5638578/
5899,SMM-менеджер,"Київ,",https://www.work.ua/jobs/5318458/
5900,Адміністратор соц. мереж,Київ,https://www.work.ua/jobs/5624384/
5901,Game Designer,Дистанційно,https://www.work.ua/jobs/3360940/


In [4]:
df2 = df['details']
df2 = df2.apply(lambda x: pd.Series(x))
df2

,salary,company,company_link,address,job_type,tags,description
0,"[9000, 19500 ]",JustSchool,https://www.work.ua/jobs/by-company/1455210/,None,"Повна зайнятість, неповна зайнятість. Також го...","[Python, Здатність до навчання, Програмування,...",
1,"[13000, 18000 ]",Megogo,https://www.work.ua/jobs/by-company/240655/,"Київ, вулиця Рибальська, 22.",Повна зайнятість. Також готові взяти студента.,"[Стресостійкість, Користувач ПК, Грамотна усна...","MEGOGO — медіасервіс для тих, хто цінує свобод..."
2,"[25000, 28000 ]","Авеон, інтернет-магазин",https://www.work.ua/jobs/by-company/2487995/,"Одеса, Овідіопольська дорога, 7.",Повна зайнятість.,"[Ведение CRM, Прием и обработка заказов]","Ми — інтернет-магазин побутових товарів, елект..."
3,None,Агропросперіс,https://www.work.ua/jobs/by-company/125534/,Київ,Повна зайнятість. Досвід роботи від 1 року.,[],
4,"[23000, 46000 ]",IPeople,https://www.work.ua/jobs/by-company/523846/,"Київ, вулиця Берковецька, 6Д.",Повна зайнятість. Досвід роботи від 1 року.,"[Комунікабельність, Впевненість, Продаж, Робот...","iPeople — це Компанія, яка обʼєднує Інновації ..."
...,...,...,...,...,...,...,...
5898,[9880 ],Галицький районний суд м. Львова,https://www.work.ua/jobs/by-company/2669586/,"Львів, вулиця Чоловського, 2.",Повна зайнятість. Також готові взяти студента....,"[MS Office, Ремонт ПК, Користувач ПК, Відповід...",Вимоги : вища освіта ступеня молодшого бакала...
5899,[25000 ],"Бережанський В.С., ФОП",https://www.work.ua/jobs/by-company/2558697/,"Київ, Велика Васильківська, 137.",Повна зайнятість. Також готові взяти студента.,"[Відповідальність, SMM, Фотозйомка, Робота в к...",Ми шукаємо творчого та відповідального менедже...
5900,"[23000, 25000 ]",Decor Interiors,https://www.work.ua/jobs/by-company/2662501/,Київ,Повна зайнятість. Також готові взяти студента.,"[Adobe Photoshop, Canva, Знання графічних реда...",Ми займаємось роздрібною торгівлею предметів д...
5901,None,Alta Games,https://www.work.ua/jobs/by-company/1434329/,None,Повна зайнятість. Досвід роботи від 2 років.,"[Unity, Ігровий дизайн, Креативність]",Ми шукаємо досвідченого Game Designer для ство...


In [5]:
df = df1.merge(df2, how='left', on=df1.index)
df.drop(columns=['key_0'], inplace=True)

In [6]:
df

,title,city,url,salary,company,company_link,address,job_type,tags,description
0,Викладач програмування (нічна зміна),Дистанційно,https://www.work.ua/jobs/5708935/,"[9000, 19500 ]",JustSchool,https://www.work.ua/jobs/by-company/1455210/,None,"Повна зайнятість, неповна зайнятість. Також го...","[Python, Здатність до навчання, Програмування,...",
1,Фахівець текстової підтримки,"Київ,",https://www.work.ua/jobs/5668550/,"[13000, 18000 ]",Megogo,https://www.work.ua/jobs/by-company/240655/,"Київ, вулиця Рибальська, 22.",Повна зайнятість. Також готові взяти студента.,"[Стресостійкість, Користувач ПК, Грамотна усна...","MEGOGO — медіасервіс для тих, хто цінує свобод..."
2,Менеджер з продажу електроніки (7 км) в інтерн...,"Одеса,",https://www.work.ua/jobs/5043547/,"[25000, 28000 ]","Авеон, інтернет-магазин",https://www.work.ua/jobs/by-company/2487995/,"Одеса, Овідіопольська дорога, 7.",Повна зайнятість.,"[Ведение CRM, Прием и обработка заказов]","Ми — інтернет-магазин побутових товарів, елект..."
3,Microsoft CRM Developer,Київ,https://www.work.ua/jobs/5613679/,None,Агропросперіс,https://www.work.ua/jobs/by-company/125534/,Київ,Повна зайнятість. Досвід роботи від 1 року.,[],
4,Продавець-консультант техніки Apple,"Київ,",https://www.work.ua/jobs/4850818/,"[23000, 46000 ]",IPeople,https://www.work.ua/jobs/by-company/523846/,"Київ, вулиця Берковецька, 6Д.",Повна зайнятість. Досвід роботи від 1 року.,"[Комунікабельність, Впевненість, Продаж, Робот...","iPeople — це Компанія, яка обʼєднує Інновації ..."
...,...,...,...,...,...,...,...,...,...,...
5898,Головний спеціаліст з інформаційних технологій,"Львів,",https://www.work.ua/jobs/5638578/,[9880 ],Галицький районний суд м. Львова,https://www.work.ua/jobs/by-company/2669586/,"Львів, вулиця Чоловського, 2.",Повна зайнятість. Також готові взяти студента....,"[MS Office, Ремонт ПК, Користувач ПК, Відповід...",Вимоги : вища освіта ступеня молодшого бакала...
5899,SMM-менеджер,"Київ,",https://www.work.ua/jobs/5318458/,[25000 ],"Бережанський В.С., ФОП",https://www.work.ua/jobs/by-company/2558697/,"Київ, Велика Васильківська, 137.",Повна зайнятість. Також готові взяти студента.,"[Відповідальність, SMM, Фотозйомка, Робота в к...",Ми шукаємо творчого та відповідального менедже...
5900,Адміністратор соц. мереж,Київ,https://www.work.ua/jobs/5624384/,"[23000, 25000 ]",Decor Interiors,https://www.work.ua/jobs/by-company/2662501/,Київ,Повна зайнятість. Також готові взяти студента.,"[Adobe Photoshop, Canva, Знання графічних реда...",Ми займаємось роздрібною торгівлею предметів д...
5901,Game Designer,Дистанційно,https://www.work.ua/jobs/3360940/,None,Alta Games,https://www.work.ua/jobs/by-company/1434329/,None,Повна зайнятість. Досвід роботи від 2 років.,"[Unity, Ігровий дизайн, Креативність]",Ми шукаємо досвідченого Game Designer для ство...
